<a href="https://colab.research.google.com/github/ScottHay14/Natural-Language-Processing-Coursework/blob/main/Natural_Language_Processing_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section 1 - Dataset

The Drug Reviews dataset from Druglib.com is a collection of patient reviews on specific drugs along with the related conditions. The dataset is broken up into these 9 variables.
<br>
<br>reviewID
<br>urlDrugName
<br>rating
<br>effectiveness
<br>sideEffects
<br>condition
<br>benefitsReview
<br>sideEffectsReview
<br>commentsReview
<br>
<br>
The task going to be performed in my classwork is text classification with the goal of predicting drug effectivness ratings from the patients reviews. The effectiveness variable is categorical and contains 5 options of effectiveness these being Highly Effective, Considerably Effective, Moderately Effective, Marginally Effective, Ineffective.



In [141]:
# Imports
import pandas as pd
import numpy as np

In [142]:
# Loading Data and combining the test and train dataset into one dataframe
test_data = "/content/Data/drugLibTest_raw.tsv"
train_data = "/content/Data/drugLibTrain_raw.tsv"

test_df = pd.read_csv(test_data, delimiter="\t")
train_df = pd.read_csv(train_data, delimiter="\t")

df = pd.concat([test_df, train_df], ignore_index=True)

In [143]:
# Exploring Data
print(df.head()) # Just printing first rows to see if loaded correctly


   Unnamed: 0 urlDrugName  rating           effectiveness  \
0        1366      biaxin       9  Considerably Effective   
1        3724    lamictal       9        Highly Effective   
2        3824    depakene       4    Moderately Effective   
3         969     sarafem      10        Highly Effective   
4         696    accutane      10        Highly Effective   

           sideEffects           condition  \
0    Mild Side Effects     sinus infection   
1    Mild Side Effects    bipolar disorder   
2  Severe Side Effects    bipolar disorder   
3      No Side Effects  bi-polar / anxiety   
4    Mild Side Effects        nodular acne   

                                      benefitsReview  \
0  The antibiotic may have destroyed bacteria cau...   
1  Lamictal stabilized my serious mood swings. On...   
2  Initial benefits were comparable to the brand ...   
3  It controlls my mood swings. It helps me think...   
4  Within one week of treatment superficial acne ...   

                   

In [144]:
# Combining the 3 review categories into one (benefitsReview, sideEffectsReview, commentsReview) for both the training dataset and the testing dataset

# Train dataset combined first
train_df["combined_review"] = train_df["benefitsReview"].fillna("").astype(str) + "\n\n" + train_df["sideEffectsReview"].fillna("").astype(str) + "\n\n" +  train_df["commentsReview"].fillna("").astype(str)
x_train = train_df["combined_review"].to_numpy()
y_train = train_df["effectiveness"].to_numpy()
print("Train dataset example")
print(x_train[0][:1000])
print(y_train[0])
print("\n")

# Test dataset combined after
test_df["combined_review"] = test_df["benefitsReview"].fillna("").astype(str) + "\n\n" + test_df["sideEffectsReview"].fillna("").astype(str) + "\n\n" +  test_df["commentsReview"].fillna("").astype(str)
x_test = test_df["combined_review"].to_numpy()
y_test = test_df["effectiveness"].to_numpy()
print("Test dataset example")
print(x_test[0][:1000])
print(y_test[0])

Train dataset example


mangagement of congestive heart failur

cough, hypotension , proteinuria, impotence , renal failure , angina pectoris , tachycardia , eosinophilic pneumonitis, tastes disturbances , anusease anorecia , weakness fatigue insominca weakness

monitor blood pressure , weight and asses for resolution of fluid
Highly Effective


Test dataset example
The antibiotic may have destroyed bacteria causing my sinus infection.  But it may also have been caused by a virus, so its hard to say.

Some back pain, some nauseau.

Took the antibiotics for 14 days. Sinus infection was gone after the 6th day.
Considerably Effective


In [145]:
# Preprocessing data
import nltk

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

def prep(X):
  prep_text = []
  for x in X:
    token_text = word_tokenize(x)
    normd_text = [token.lower() for token in token_text if token.isalpha()]
    swr_text = [token for token in normd_text if token not in stopwords.words("english")]
    stemmer = SnowballStemmer("english")
    prep_text += [[stemmer.stem(word) for word in swr_text]]
  prep_sentences = [" ".join(sentence) for sentence in prep_text]
  return prep_sentences

prep_x_train = prep(x_train)
prep_x_test = prep(x_test)

print("Preprocessed working for train dataset")
print(prep_x_train[0][:1000])

print("Preprocessed working for test dataset")
print(prep_x_test[0][:1000])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Preprocessed working for train dataset
slow progress left ventricular dysfunct overt heart failur alon agent manag hypertens mangag congest heart failur cough hypotens proteinuria impot renal failur angina pectori tachycardia eosinophil pneumon tast disturb anuseas anorecia weak fatigu insominca weak monitor blood pressur weight ass resolut fluid
Preprocessed working for test dataset
antibiot may destroy bacteria caus sinus infect may also caus virus hard say back pain nauseau took antibiot day sinus infect gone day


## Section 2 - Representation Learning

-

In [149]:
# pip install to get the Word2Vec to work
!pip install gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [160]:
# First Representation Learning - Word2Vec
def word2vec_rep(sentence, w2v_model):
  embs = [w2v_model.wv[word] for word in sentence if word in w2v_model.wv.index_to_key]
  if len(embs) > 0:
    sent_emb = np.mean(np.array(embs), axis=0)
  else:
    sent_emb = np.zeros(w2v_model.vector_size)
  return sent_emb

prep_x_train_token = [sentence.split() for sentence in prep_x_train]
prep_x_test_token = [sentence.split() for sentence in prep_x_test]

w2v_model = Word2Vec(vector_size=300, window=5, min_count=2, workers=4, sg=0)
w2v_model.build_vocab(prep_x_train_token, update=None)

w2v_model.train(
    corpus_iterable=prep_x_train_token,
    total_examples=len(prep_x_train_token),
    epochs=10
)

print(f"Vocabulary: {len(w2v_model.wv.index_to_key)}")
print(f"Sample words: {list(w2v_model.wv.index_to_key)[:10]}")

x_train_representation = [word2vec_rep(instance, w2v_model) for instance in prep_x_train_token]
x_test_representation = [word2vec_rep(instance, w2v_model) for instance in prep_x_test_token]

x_train_representation = np.array(x_train_representation)
x_test_representation = np.array(x_test_representation)


# Data Exploration
print("Words that are similar to 'Sick'",w2v_model.wv.most_similar("sick", topn=3))
print("Words that are similar to 'Benefit'",w2v_model.wv.most_similar("benefit", topn=3))

Vocabulary: 4966
Sample words: ['take', 'day', 'effect', 'side', 'time', 'drug', 'pain', 'medic', 'week', 'feel']
Words that are similar to 'Sick' [('nauseous', 0.9322900176048279), ('flu', 0.9219678044319153), ('ate', 0.914432942867279)]
Words that are similar to 'Benefit' [('outweigh', 0.903175950050354), ('negat', 0.8546966910362244), ('ignific', 0.8426893949508667)]


In [159]:
# Second Representation Learning - TF-IDF
tfidf = TfidfVectorizer(max_features=1000)
x_train_tfidf = tfidf.fit_transform(prep_x_train).toarray()
x_test_tfidf = tfidf.fit_transform(prep_x_test).toarray()

# Data Exploration
feature_names = tfidf.get_feature_names_out()
print(f"Sample {list(feature_names)[:15]}")


Sample ['abdomin', 'abil', 'abl', 'absolut', 'accutan', 'ach', 'achiev', 'acid', 'acn', 'activ', 'actual', 'acut', 'acyclovir', 'ad', 'add']


## Section 3 - Algorithms

-

## Section 4 - Evaluation